# Fine-tuning GPT2

In [1]:
import torch

from transformers import CONFIG_MAPPING, AutoTokenizer, \
                         AutoModelWithLMHead, TextDataset, \
                         DataCollatorForLanguageModeling, \
                         Trainer, TrainingArguments

from os.path import join

In [2]:
!ls ../datasets/WikiText-2-raw

cached_lm_GPT2Tokenizer_1024_wiki.train.raw  wiki.test.raw   wiki.valid.raw
cached_lm_GPT2Tokenizer_1024_wiki.valid.raw  wiki.train.raw


In [3]:
!cat ../datasets/WikiText-2-raw/wiki.test.raw | tail

 " Eddie , you 're a born loser . " - Nominated 
 AFI 's 100 Years ... 100 Movies ( 10th Anniversary Edition ) - Nominated 
 AFI 's 10 Top 10 - # 6 Sports Film 
 
 = = Legacy = = 
 
 In the decades since its release , The Hustler has cemented its reputation as a classic . Roger Ebert , echoing earlier praise for the performances , direction , and cinematography and adding laurels for editor Dede Allen , cites the film as " one of those films where scenes have such psychic weight that they grow in our memories . " He further cites Fast Eddie Felson as one of " only a handful of movie characters so real that the audience refers to them as touchstones . " TV Guide calls the film a " dark stunner " offering " a grim world whose only bright spot is the top of the pool table , yet [ with ] characters [ who ] maintain a shabby nobility and grace . " The four leads are again lavishly praised for their performances and the film is summed up as " not to be missed . " 
 Paul Newman reprised his r

In [4]:
DATASET_FOLDER = '../datasets/WikiText-2-raw'
TRAIN_FILE = join(DATASET_FOLDER, 'wiki.train.raw')
VALID_FILE = join(DATASET_FOLDER, 'wiki.valid.raw')
TEST_FILE = join(DATASET_FOLDER, 'wiki.test.raw')

## Obtener dataset

In [5]:
def get_dataset(tokenizer, evaluate=False):
    file_path = VALID_FILE if evaluate else TRAIN_FILE
    block_size = tokenizer.max_len
    return TextDataset(tokenizer=tokenizer,
                       file_path=file_path,
                       block_size=block_size,
                       overwrite_cache=False
                      )


In [6]:
config = CONFIG_MAPPING['gpt2']()

In [7]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')

In [8]:
model = AutoModelWithLMHead.from_pretrained('gpt2',
            config=config,
        )

In [9]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [10]:
train_dataset = get_dataset(tokenizer=tokenizer)
eval_dataset = get_dataset(tokenizer=tokenizer, evaluate=True)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [12]:
from os.path import isdir

output_dir = "./output-dir2"
if isdir(output_dir):
    raise ValueError(f"{output_dir} folder already exists")

logging_dir = "./testing_logging2"
if isdir(logging_dir):
    raise ValueError(f"{logging_dir} folder already exists")

training_args = TrainingArguments(output_dir,
                                  logging_dir=logging_dir,
                                  per_gpu_train_batch_size=1,
                                  per_gpu_eval_batch_size=1)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    prediction_loss_only=True,
)

In [14]:
 # Training
model_path = None

trainer.train(model_path=model_path)
trainer.save_model()
# For convenience, we also re-save the tokenizer to the same directory,
# so that you can share your model easily on huggingface.co/models =)
if trainer.is_world_master():
    tokenizer.save_pretrained(output_dir)


/home/gschuit/.pyenv/versions/3.6.8/envs/huggingface/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


{"learning_rate": 4.294383290996331e-05, "loss": 3.244473777294159, "step": 500}


/home/gschuit/.pyenv/versions/3.6.8/envs/huggingface/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{"learning_rate": 3.588766581992662e-05, "loss": 3.1490229716300964, "step": 1000}



{"learning_rate": 2.883149872988993e-05, "loss": 3.0468226017951965, "step": 1500}
{"learning_rate": 2.1775331639853235e-05, "loss": 2.9714888110160826, "step": 2000}



{"learning_rate": 1.4719164549816541e-05, "loss": 2.9544080247879028, "step": 2500}
{"learning_rate": 7.66299745977985e-06, "loss": 2.8888942613601682, "step": 3000}
{"learning_rate": 6.068303697431555e-07, "loss": 2.8971941576004028, "step": 3500}




In [ ]:
# Evaluation
results = {}
logger.info("*** Evaluate ***")

eval_output = trainer.evaluate()

perplexity = math.exp(eval_output["eval_loss"])
result = {"perplexity": perplexity}

output_eval_file = os.path.join(training_args.output_dir, "eval_results_lm.txt")
if trainer.is_world_master():
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results *****")
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

results.update(result)

In [26]:
 # Training
model_path = None

trainer.train(model_path=model_path)
trainer.save_model()
# For convenience, we also re-save the tokenizer to the same directory,
# so that you can share your model easily on huggingface.co/models =)
if trainer.is_world_master():
    tokenizer.save_pretrained(output_dir)

{"learning_rate": 4.294383290996331e-05, "loss": 2.8090043258666992, "step": 500}




KeyboardInterrupt: 